In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import helper

## LSTM GAN Training

### PAMAP2

In [ ]:
from DeepConvLSTM_model import DeepConvNet
val_model = helper.load_pl_model("val_model.ckpt", DeepConvNet, hidden_size = 256)
helper.train_LSTM_GAN(helper.load_PAMAP2_activity,
                     val_model,
                     start_activity = 1,
                     total_activities = 7,
                     val_iter_size = 3, 
                     batch_size = 20,
                     data_size = (27, 100),
                     noise_len = 100,
                     gen_num_layers = 2,
                     gen_bidirectional = False,
                     dis_hidden_size = 100,
                     dis_num_layers = 1,
                     dis_bidirectional = False,
                     tensorboard_save_dir = "LSTM_GAN_logs",
                     tensorboard_name_prefix = "PAMAP2_act_",
                     )

### RWHAR

In [ ]:
from TransformerClassifier import TransformerClassifier
val_model = helper.load_pl_model("RWHAR_transformer.ckpt", TransformerClassifier, 
                                 in_channels = 6, output_size = 8, d_model = 50, nhead = 5, 
                                 dim_feedforward = 10000, num_layers = 5)
helper.train_LSTM_GAN(helper.load_RWHAR_activity,
                     val_model,
                     start_activity = 1,
                     total_activities = 8,
                     val_iter_size = 3, 
                     batch_size = 128,
                     data_size = (6, 50),
                     noise_len = 50,
                     gen_num_layers = 3,
                     gen_bidirectional = True,
                     dis_hidden_size = 50,
                     dis_num_layers = 3,
                     dis_bidirectional = True,
                     decay = 0,
                     tensorboard_save_dir = "LSTM_GAN_logs",
                     tensorboard_name_prefix = "RWHAR_act_",
                     )


## Transformer GAN
### PAMAP2

In [ ]:
from TransformerClassifier import TransformerClassifier
val_model = helper.load_pl_model("val_model_transformer.ckpt", TransformerClassifier, 
                                 in_channels = 27, output_size = 7, 
                                 d_model = 100, nhead = 1, dim_feedforward = 2048,)
helper.train_transformer_GAN(
    helper.load_PAMAP2_activity,
    val_model,
    start_activity = 1,
    total_activities = 7,
    val_iter_size = 3,
    batch_size = 32,
    data_size = (27, 100),
    noise_len = 100,
    period = 100,
    max_retries = 2,
    init_dim_feedforward = 2048,
    dim_feedforward_exponent = 5,
    gen_nheads = 5,
    dis_nheads = 5,
    tensorboard_save_dir = "transformer_GAN_logs",
    tensorboard_name_prefix = "PAMAP2_act_",
    )

### RWHAR

In [ ]:
from TransformerClassifier import TransformerClassifier
val_model = helper.load_pl_model("RWHAR_transformer.ckpt", TransformerClassifier, 
                                 in_channels = 6, output_size = 8, d_model = 50, nhead = 5, 
                                 dim_feedforward = 10000, num_layers = 5)
helper.train_transformer_GAN(
    helper.load_RWHAR_activity,
    val_model,
    start_activity = 1,
    total_activities = 8,
    val_iter_size = 3,
    batch_size = 128,
    data_size = (6, 50),
    noise_len = 50,
    period = 100,
    max_retries = 2,
    init_dim_feedforward = 2048,
    dim_feedforward_exponent = 5,
    gen_nheads = 5,
    dis_gen_heads = 5,
    tensorboard_save_dir = "transformer_GAN_logs",
    tensorboard_name_prefix = "RWHAR_act_",
    )

In [ ]:
## Things to Try to stabilise GAN
## 3. Use one sided label smoothing for discriminator (helps a little)
## 5. Remove Linear and use conv only (helps a ton)
## 6. Use dropout of 0.5 